# 02 - Silver Layer: Enrich with Jikan API

In [0]:
import requests
import json
import time
import os
import sys
from pyspark.sql import functions
import pandas as pd

In [0]:
Silver_table = "anime_project.silver.anime_enriched"
cache_path = "/Volumes/anime_project/cache/anime_cache/anime_cache.json"
API_rate_limit = 0.7
anime_data = "/Volumes/anime_project/bronze/anime_data"

Loading bronze layer data

In [0]:
df_bronze = spark.read.format("delta").load("/Volumes/anime_project/bronze/anime_data")

In [0]:
df_bronze.explain()

In [0]:
print(f"   Total rows: {df_bronze.count()}")
print(f"   Total columns: {len(df_bronze.columns)}")

Show columns

In [0]:
df_bronze.columns

Sample data

In [0]:
print("\n👀 Sample data:")
display(df_bronze.limit(5))

Select Only Needed Columns

In [0]:
columns_needed = [
    'user_id',           # Keep for multi-user tracking
    'user_name',         # Keep for multi-user tracking
    'series_animedb_id', # Anime ID (for API calls)
    'series_title',      # Anime title
    'my_score',          # Your rating
    'my_status',         # Completed/Watching/etc
    'series_type'        # TV/Movie/OVA/etc
]

In [0]:
df_selected = df_bronze.select(columns_needed)

print(f"✅ Selected {len(columns_needed)} columns from {len(df_bronze.columns)} total")
print(f"\n📋 Columns kept:")
for col in columns_needed:
    print(f"    - {col}")

Convert to Pandas for API enrichment

In [0]:
df_pandas = df_selected.toPandas()

print(f"\n📊 Pandas DataFrame:")
print(f"   Shape: {df_pandas.shape}")

In [0]:
df_pandas.rename(columns={
    'series_animedb_id': 'anime_id',
    'series_title': 'anime_title',
    'my_score': 'user_score',
    'my_status': 'status'
}, inplace=True)

In [0]:
print(f"\n📈 Status Distribution:")
print(df_pandas['status'].value_counts())

In [0]:
print(f"\n👀 Sample data:")
display(df_pandas.head(10))

Define Jikan API Function (with Caching)

In [0]:
def get_anime_info(anime_id, cache_file=cache_path):
    """
    Fetch anime info from Jikan API v4, with local caching.
    
    Args:
        anime_id: MAL anime ID (integer)
        cache_file: Path to JSON cache file
    
    Returns:
        Dictionary with anime data, or None if failed
    """
    # Load cache if it exists
    if os.path.exists(cache_file):
        with open(cache_file, "r", encoding="utf-8") as f:
            cache = json.load(f)
    else:
        cache = {}
    
    # Check if already cached
    if str(anime_id) in cache:
        return [cache[str(anime_id)], True]
    
    # Fetch from API
    url = f"https://api.jikan.moe/v4/anime/{anime_id}/full"
    
    try:
        response = requests.get(url, timeout=10)
        
        if response.status_code == 200:
            anime = response.json()["data"]
            
            # Extract the fields we want
            info = {
                "id": anime["mal_id"],
                "synopsis": anime["synopsis"],
                "genres": [g["name"] for g in anime["genres"]],
                # "explicit_genres": [eg["name"] for eg in anime["explicit_genres"]], (All Null Values)
                "themes": [t["name"] for t in anime["themes"]],
                "studios": [s["name"] for s in anime["studios"]],
                "score": anime["score"],
                "popularity": anime["popularity"],
                "source": anime["source"],
                "rating": anime["rating"],
                "season": anime["season"],
                "year": anime["year"]
            }
            
            # Save to cache
            cache[str(anime_id)] = info
            with open(cache_file, "w", encoding="utf-8") as f:
                json.dump(cache, f, indent=2, ensure_ascii=False)
            
            return [info, False]
        
        elif response.status_code == 429:
            # Rate limit hit
            print(f"\n⚠️ Rate limit hit, waiting 60s...")
            time.sleep(60)
            return get_anime_info(anime_id, cache_file)  # Retry
        
        elif response.status_code == 404:
            print(f"\n⚠️ Anime {anime_id} not found (404)")
            return None
        
        else:
            print(f"\n⚠️ HTTP {response.status_code} for anime {anime_id}")
            return None
    
    except Exception as e:
        print(f"\n❌ Error fetching {anime_id}: {str(e)}")
        return None

print("✅ API function defined: get_anime_info()")
print(f"   Cache location: {cache_path}")

Testing function

In [0]:
test_anime_id = df_pandas.iloc[0]['anime_id']
test_anime_title = df_pandas.iloc[0]['anime_title']

print(f"🧪 Testing API with:")
print(f"   Anime ID: {test_anime_id}")
print(f"   Title: {test_anime_title}")

In [0]:
test_result, cache = get_anime_info(test_anime_id)

if test_result:
    print(f"\n✅ API call successful!")
    print(f"\n📋 Data retrieved:")
    for key, value in test_result.items():
        # Truncate long strings for display
        if isinstance(value, str) and len(value) > 100:
            print(f"   {key}: {value[:100]}...")
        else:
            print(f"   {key}: {value}")
else:
    print(f"\n❌ API call failed!")

Enrich All Anime (with Progress Bar)


In [0]:
def enrich_dataset(user_df, get_info_fn, rate_limit=0.8):
    """
    Enrich user dataset with metadata using API + caching.
    Shows progress bar.
    
    Args:
        user_df: Pandas DataFrame with anime_id column
        get_info_fn: Function to fetch anime info
        rate_limit: Seconds to wait between API calls
    
    Returns:
        Enriched Pandas DataFrame
    """
    extra_info = []
    total = len(user_df)
    
    print(f"🚀 Starting enrichment for {total} anime...")
    print(f"   Rate limit: {rate_limit}s per request")
    print(f"   Estimated time: ~{int(total * rate_limit / 60)} minutes\n")

    for idx, row in user_df.iterrows():
        anime_id = row['anime_id']
        
        # Fetch info
        info, cache = get_info_fn(anime_id)
        
        if info:
            # Normalize list fields to comma-separated strings
            for key in ["genres", "themes", "studios", "explicit_genres"]:
                if key in info and isinstance(info[key], list):
                    info[key] = ", ".join(info[key]) if info[key] else None
            
            extra_info.append(info)
        
        # Progress bar (simple text-based)
        progress = int((idx + 1) / total * 50)  # 50 chars wide
        sys.stdout.write(
            f"\r[{'#' * progress}{'.' * (50 - progress)}] {idx + 1}/{total}"
        )
        sys.stdout.flush()
        
        # Rate limiting
        if cache == False: 
            time.sleep(rate_limit)
    
    print("\n\n✅ Enrichment complete!")
    
    # Merge with original dataframe
    extra_df = pd.DataFrame(extra_info)
    enriched_df = user_df.merge(
        extra_df, 
        left_on="anime_id", 
        right_on="id", 
        how="left"
    ).drop(columns=["id"])
    
    return enriched_df

In [0]:
print("=" * 70)
enriched_dataset = enrich_dataset(df_pandas, get_anime_info, API_rate_limit)
print("=" * 70)

print(f"\n📊 Enrichment Results:")
print(f"   Original columns: {len(df_pandas.columns)}")
print(f"   Enriched columns: {len(enriched_dataset.columns)}")
print(f"   New fields added: {len(enriched_dataset.columns) - len(df_pandas.columns)}")

Verify Enriched Data

In [0]:
print("=" * 70)
print("📊 ENRICHED DATASET VERIFICATION")
print("=" * 70)

print(f"\n✅ Shape: {enriched_dataset.shape}")
print(f"   Rows: {enriched_dataset.shape[0]}")
print(f"   Columns: {enriched_dataset.shape[1]}")

print(f"\n📋 All Columns:")
for i, col in enumerate(enriched_dataset.columns, 1):
    print(f"   {i:2d}. {col}")

In [0]:
print(f"\n🔍 Data Completeness (NULL counts):")
new_cols = ['synopsis', 'genres', 'explicit_genres', 'themes', 'studios', 'score', 'popularity', 'source', 'rating', 'season', 'year']
for col in new_cols:
    if col in enriched_dataset.columns:
        null_count = enriched_dataset[col].isna().sum()
        null_pct = (null_count / len(enriched_dataset)) * 100
        print(f"   {col:20s}: {null_count:3d} nulls ({null_pct:.1f}%)")

In [0]:
print(f"\n👀 Sample Enriched Data:")
display(enriched_dataset.head(10))

In [0]:
print(f"\n⭐ Score Comparison:")
scored = enriched_dataset[enriched_dataset['user_score'].notna() & enriched_dataset['score'].notna()]
if len(scored) > 0:
    print(f"   Entries with both scores: {len(scored)}")
    print(f"   Your average score: {scored['user_score'].mean():.2f}")
    print(f"   MAL average score: {scored['score'].mean():.2f}")
    print(f"   Difference: {scored['user_score'].mean() - scored['score'].mean():.2f}")

Save to Silver Layer (Delta Table)

In [0]:
df_silver = spark.createDataFrame(enriched_dataset)

print("✅ Converted to Spark DataFrame")
print("\n📋 Schema:")
df_silver.printSchema()

In [0]:
df_silver = df_silver \
    .withColumn("processing_timestamp", functions.current_timestamp()) \
    .withColumn("processing_date", functions.current_date()) \
    .withColumn("data_source", functions.lit("jikan_api_enrichment"))

print("\n✅ Created plan metadata columns:")
print("   - processing_timestamp")
print("   - processing_date")
print("   - data_source")

In [0]:
print(f"\n💾 Saving to Silver table: {Silver_table}")

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(Silver_table)

In [0]:
%sql
SHOW TABLES IN anime_project.silver

In [0]:
%sql
SELECT user_id, user_name, anime_id, anime_title, user_score, status, series_type, synopsis, genres, themes, studios, score, popularity, source, rating, season, year, processing_timestamp, processing_date, data_source FROM anime_project.silver.anime_enriched LIMIT 10;